In [ ]:
from langchain_community.cache import AzureCosmosDBSemanticCache
from langchain.globals import set_llm_cache
import urllib 

AzureCosmosDBSemanticCache
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)

from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain_openai import OpenAI, AzureOpenAI
from langchain.globals import set_llm_cache

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential

from dotenv import load_dotenv

In [ ]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "example.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

COSMOS_MONGO_USER = config['cosmos_db_mongo_user']
COSMOS_MONGO_PWD = config['cosmos_db_mongo_pwd']
COSMOS_MONGO_SERVER = config['cosmos_db_mongo_server']          

In [ ]:
INDEX_NAME = "VectorSearchIndex"
NAMESPACE = "ExampleDB.CachingCollection"

CONNECTION_STRING = ("mongodb+srv://"+COSMOS_MONGO_USER+":"+COSMOS_MONGO_PWD+"@"+COSMOS_MONGO_SERVER+"?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000")

DB_NAME, COLLECTION_NAME = NAMESPACE.split(".")

In [ ]:
# Requires model version 0301 or more recent
llm = AzureOpenAI(
    deployment_name=config['openai_completions_deployment'],
    model_name=config['openai_completions_model'],
    openai_api_key=config['openai_api_key'],
    azure_endpoint=config['openai_api_endpoint'],
    openai_api_version=config['openai_api_version'],
    n=2,
    best_of=2,
    cache=True)


embeddings = AzureOpenAIEmbeddings(
    azure_deployment=config['openai_embeddings_deployment'],
    model=config['openai_embeddings_model'],
    openai_api_key=config['openai_api_key'],
    azure_endpoint=config['openai_api_endpoint'],
    openai_api_version=config['openai_api_version'])

In [ ]:
# Default value for these params
num_lists = 1
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_IVF
m = 16
ef_construction = 64
ef_search = 40
score_threshold = 0.9

set_llm_cache(
    AzureCosmosDBSemanticCache(
        cosmosdb_connection_string=CONNECTION_STRING,
        cosmosdb_client=None,
        embedding=embeddings,
        database_name=DB_NAME,
        collection_name=COLLECTION_NAME,
        num_lists=num_lists,
        similarity=similarity_algorithm,
        kind=kind,
        dimensions=dimensions,
        m=m,
        ef_construction=ef_construction,
        ef_search=ef_search,
        score_threshold=score_threshold    )
)

In [ ]:
%%time
# The first time, the quesiton/response is not yet cachced in Cosmos DB, so retrieval should be slower
llm("Tell me about beer making")

In [ ]:
%%time
# This quesiton/response is not yet cachced in Cosmos DB, so retrieval should be slower
llm("Tell me a joke?")

In [ ]:
%%time
# The second time, the quesiton/response is cached in Cosmos DB, so retrieval should be faster
llm("Tell me about beer making")

In [ ]:
%%time
# This question is semantically similar to the previous one within the score_threshold amount, so retrieval should be faster
llm("How do I make beer?")